In [1]:
%pip install pandas torch numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
df = pd.read_csv("data.csv")

In [ ]:
import soundata
import numpy as np
np.float_ = np.float64

dataset = soundata.initialize('urbansound8k')
dataset.download()
dataset.validate()

INFO: Downloading ['all', 'index']. Index is being stored in /home/guus/vscodeProjects/api-multi-label-classification/.venv/lib/python3.10/site-packages/soundata/datasets/indexes, and the rest of files in /home/guus/sound_datasets/urbansound8k
INFO: [all] downloading UrbanSound8K.tar.gz
INFO: /home/guus/sound_datasets/urbansound8k/UrbanSound8K.tar.gz already exists and will not be downloaded. Rerun with force_overwrite=True to delete this file and force the download.
INFO: /home/guus/sound_datasets/urbansound8k/UrbanSound8K_README.txt already exists. Run with force_overwrite=True to download from scratch
INFO: /home/guus/sound_datasets/urbansound8k/metadata already exists. Run with force_overwrite=True to download from scratch
INFO: /home/guus/sound_datasets/urbansound8k/FREESOUNDCREDITS.txt already exists. Run with force_overwrite=True to download from scratch
INFO: /home/guus/sound_datasets/urbansound8k/audio already exists. Run with force_overwrite=True to download from scratch
INFO

({'metadata': {}, 'clips': {}}, {'metadata': {}, 'clips': {}})

: 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df['Fold'] = df['AudioPath'].apply(lambda x: int(x.split('/')[-2].replace('fold', '')))

spectrograms = np.array(df['Spectrogram'].tolist())
lengths = np.array([len(s) for s in spectrograms])
max_length = max(lengths)

X = np.array([np.pad(s, (0, max_length - len(s)), mode='constant') for s in spectrograms])
y = np.array([clip.tags.labels[0] for key, clip in dataset.load_clips().items()])

accuracies = []

for fold in range(1, 11):
    print(f'Fold {fold}')
    train_index = df[df['Fold'] != fold].index
    test_index = df[df['Fold'] == fold].index

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

print(f'Mean accuracy: {np.mean(accuracies)}')

Fold 1
